### Getting data

Query to retrive the data("df" in the following Notebook):

In [388]:
# select A.ID, A.KEYS:src:keys:region as region,A.KEYS:src:keys:accountcallee as accountcallee,
#     A.KEYS:src:keys:accountcaller as accountcaller,A.KEYS:src:keys:api as api,A.KEYS:src:keys:calltype as calltype,
#     A.KEYS:src:keys:recipientAccountAlias as recipientAccountAlias,A.KEYS:src:keys:service as service,A.KEYS:src:keys:source as source,A.start_time, A.end_time,
#     A.KEYS:src:keys:username as username,
#     B.ID as ID_B, B.AWS_REGION, B.EVENT_NAME --A.start_time, A.event_type, 
#     from EVENT_INTERNAL.EVENT_DETAILS_T A
#     join cloud_trail_internal.event_summary_t B
#     -- have to do a CAST to join the two tables
#     on cast(A.KEYS:src:keys:Id AS int) = B.id and A.start_time = B.start_time
#     where A.start_time between to_timestamp('05/01/2019 12:00:00') and to_timestamp('07/01/2019 12:00:00') and props:model = 'AwsApiTracker';

In [389]:
import pandas as pd
import numpy as np
from collections import Counter

In [390]:
df = pd.read_csv("/users/tianhaoluo/Downloads/result-may-june.csv")

In [391]:
type(df['START_TIME'][0])

str

In [392]:
df['START_TIME'] = df['START_TIME'].astype(str)
df['END_TIME'] = df['END_TIME'].astype(str)

In [393]:
df.head()

,ID,REGION,ACCOUNTCALLEE,ACCOUNTCALLER,API,CALLTYPE,RECIPIENTACCOUNTALIAS,SERVICE,SOURCE,START_TIME,END_TIME,USERNAME,ID_B,AWS_REGION,EVENT_NAME
0,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketAcl""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.28.200""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529589,us-west-2,GetBucketAcl
1,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPublicAccessBlock""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.25.120""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529590,us-west-2,GetBucketPublicAccessBlock
2,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPolicyStatus""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.28.200""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529591,us-west-2,GetBucketPolicyStatus
3,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPolicyStatus""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.25.120""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529592,us-west-2,GetBucketPolicyStatus
4,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPublicAccessBlock""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.28.200""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529594,us-west-2,GetBucketPublicAccessBlock


In [394]:
np.array(df.loc[df['ID'] == 2796]['API'])

array(['GetBucketAcl""', 'GetBucketPublicAccessBlock""',
       'GetBucketPolicyStatus""', 'GetBucketPolicyStatus""',
       'GetBucketPublicAccessBlock""', 'GetBucketPolicyStatus""',
       'GetBucketAcl""', 'GetBucketPublicAccessBlock""',
       'GetBucketPublicAccessBlock""', 'GetBucketAcl""', 'GetBucketAcl""',
       'GetBucketPolicyStatus""', 'GetBucketPublicAccessBlock""',
       'GetBucketPolicyStatus""', 'GetBucketPolicyStatus""',
       'GetBucketPublicAccessBlock""', 'GetBucketPolicyStatus""',
       'GetBucketPublicAccessBlock""', 'GetBucketPolicyStatus""',
       'GetBucketAcl""', 'GetBucketPublicAccessBlock""', 'GetBucketAcl""',
       'GetBucketPublicAccessBlock""', 'GetBucketPolicyStatus""',
       'GetBucketPolicyStatus""', 'GetBucketPublicAccessBlock""',
       'GetBucketPolicyStatus""', 'GetBucketAcl""',
       'GetBucketPublicAccessBlock""', 'GetBucketPublicAccessBlock""',
       'GetBucketAcl""', 'GetBucketAcl""', 'GetBucketPolicyStatus""',
       'GetBucketPublicAc

In [423]:
Counter(np.array(df.loc[df['ID'] == 2796]['API']))

Counter({'GetBucketAcl""': 12,
         'GetBucketPublicAccessBlock""': 18,
         'GetBucketPolicyStatus""': 18})

In [395]:
list(df)

['ID',
 'REGION',
 'ACCOUNTCALLEE',
 'ACCOUNTCALLER',
 'API',
 'CALLTYPE',
 'RECIPIENTACCOUNTALIAS',
 'SERVICE',
 'SOURCE',
 'START_TIME',
 'END_TIME',
 'USERNAME',
 'ID_B',
 'AWS_REGION',
 'EVENT_NAME']

In [396]:
entity_list = ['REGION',
 'ACCOUNTCALLEE',
 'ACCOUNTCALLER',
 'API',
 'CALLTYPE',
 'RECIPIENTACCOUNTALIAS',
 'SERVICE',
 'SOURCE',
 'USERNAME']

In [397]:
#id_val = -1 means include results from all IDs
def getData(df,entity_name, id_val = -1, start_time = "2019-06-12", end_time = "2019-06-13"):
    if id_val < 0:
        return np.array(df.loc[(df['START_TIME'] >= start_time) &  (df['START_TIME'] < end_time)][entity_name])
    return np.array(df.loc[(df['START_TIME'] >= start_time) &  (df['START_TIME'] < end_time) & (df["ID"] == id_val)][entity_name])

In [398]:
col = getData(df,'REGION')
Counter(col)

Counter({'us-west-2""': 32, 'us-east-1""': 158, 'us-west-1""': 1})

In [399]:
def getEntropy(col):
    cnt_dict = Counter(col)
    counts = np.array([v for k,v in cnt_dict.items()])
    total_counts = sum(counts)
    probs = [c/total_counts for c in counts]
    return -np.sum(probs*np.log(probs))

In [400]:
def getUnlikeability(col):
    cnt_dict = Counter(col)
    counts = np.array([v for k,v in cnt_dict.items()])
    N = sum(counts)
    pairs_same_val = sum([n*(n-1)/2 for n in counts])
    pairs_total  = 1/2*N**2-1/2*N
    #print([n*(n-1)/2 for n in counts],pairs_total)
    return 1-pairs_same_val/pairs_total

In [401]:
# Counter(col)

In [402]:
# getUnlikeability([1,1,2,2])

In [403]:
entity_list = ['REGION',
 'ACCOUNTCALLEE',
 'ACCOUNTCALLER',
 'API',
 'CALLTYPE',
 'RECIPIENTACCOUNTALIAS',
 'SERVICE',
 'SOURCE',
 'USERNAME']

In [404]:
def getWeights(df,entity_list,id_val=-1,method="entropy",start_time = "2019-06-12", end_time = "2019-06-13", normalized = True):
    d = {}
    for entity_name in entity_list:
        col = getData(df, entity_name, id_val, start_time, end_time)
        if method == "entropy":
            d[entity_name] = getEntropy(col)
        elif method == "unlikeability":
            d[entity_name] = getUnlikeability(col)
    if not normalized:
        return d
    w_sum = sum([v for k,v in d.items()])
    for entity_name in entity_list:
        d[entity_name] /= w_sum
    return d

### Data for June 12

In [405]:
df[(df['START_TIME'] > "2019-06-12") & (df['START_TIME'] < "2019-06-13") ][['ID','ID_B']+entity_list].head(50)

,ID,ID_B,REGION,ACCOUNTCALLEE,ACCOUNTCALLER,API,CALLTYPE,RECIPIENTACCOUNTALIAS,SERVICE,SOURCE,USERNAME
5437,4440,43233242,"us-west-2""""","249446771485""""","249446771485""""","DescribeDBClusterSnapshots""""","AwsApiCall""""","lacework-test""""","rds.amazonaws.com""""","35.163.131.126""""","AssumedRole/249446771485:tf-role-qa2-worker"""""
5438,4436,43230607,"us-east-1""""","249446771485""""","249446771485""""","ListAccountAliases""""","AwsApiCall""""","lacework-test""""","iam.amazonaws.com""""","182.72.79.66""""","IAMUser/249446771485:dev_saurabh"""""
5439,4434,43230607,"us-east-1""""","249446771485""""","249446771485""""","ListAccountAliases""""","AwsApiCall""""","lacework-test""""","iam.amazonaws.com""""","182.72.79.66""""","IAMUser/249446771485:dev_saurabh"""""
5440,4434,43230608,"us-east-1""""","249446771485""""","249446771485""""","ListAccessKeys""""","AwsApiCall""""","lacework-test""""","iam.amazonaws.com""""","182.72.79.66""""","IAMUser/249446771485:dev_saurabh"""""
5441,4436,43230608,"us-east-1""""","249446771485""""","249446771485""""","ListAccessKeys""""","AwsApiCall""""","lacework-test""""","iam.amazonaws.com""""","182.72.79.66""""","IAMUser/249446771485:dev_saurabh"""""
5442,4436,43230609,"us-east-1""""","249446771485""""","249446771485""""","GetAccountSummary""""","AwsApiCall""""","lacework-test""""","iam.amazonaws.com""""","182.72.79.66""""","IAMUser/249446771485:dev_saurabh"""""
5443,4434,43230609,"us-east-1""""","249446771485""""","249446771485""""","GetAccountSummary""""","AwsApiCall""""","lacework-test""""","iam.amazonaws.com""""","182.72.79.66""""","IAMUser/249446771485:dev_saurabh"""""
5444,4436,43230610,"us-east-1""""","249446771485""""","249446771485""""","GetAccountPasswordPolicy""""","AwsApiCall""""","lacework-test""""","iam.amazonaws.com""""","182.72.79.66""""","IAMUser/249446771485:dev_saurabh"""""
5445,4434,43230610,"us-east-1""""","249446771485""""","249446771485""""","GetAccountPasswordPolicy""""","AwsApiCall""""","lacework-test""""","iam.amazonaws.com""""","182.72.79.66""""","IAMUser/249446771485:dev_saurabh"""""
5446,4433,43229668,"us-east-1""""","249446771485""""","249446771485""""","GetUser""""","AwsApiCall""""","lacework-test""""","iam.amazonaws.com""""","114.143.149.14""""","IAMUser/249446771485:dev_swapnil"""""


In [406]:
getWeights(df,entity_list,method="entropy",normalized = False)

{'REGION': 0.48372073638248325,
 'ACCOUNTCALLEE': -0.0,
 'ACCOUNTCALLER': -0.0,
 'API': 3.3903809428453036,
 'CALLTYPE': 0.3353147179260296,
 'RECIPIENTACCOUNTALIAS': -0.0,
 'SERVICE': 1.0587188236076357,
 'SOURCE': 1.418389022567496,
 'USERNAME': 2.0522736518282434}

In [407]:
getWeights(df,entity_list,method="entropy",normalized = True)

{'REGION': 0.05535323532891731,
 'ACCOUNTCALLEE': -0.0,
 'ACCOUNTCALLER': -0.0,
 'API': 0.38796880114645543,
 'CALLTYPE': 0.03837080591048479,
 'RECIPIENTACCOUNTALIAS': -0.0,
 'SERVICE': 0.12115154009847848,
 'SOURCE': 0.16230939765222507,
 'USERNAME': 0.23484621986343898}

In [408]:
getWeights(df,entity_list,method="unlikeability",normalized = False)

{'REGION': 0.2891154588040783,
 'ACCOUNTCALLEE': 0.0,
 'ACCOUNTCALLER': 0.0,
 'API': 0.962524111325434,
 'CALLTYPE': 0.18848167539267013,
 'RECIPIENTACCOUNTALIAS': 0.0,
 'SERVICE': 0.4629374483328741,
 'SOURCE': 0.7167814825020666,
 'USERNAME': 0.8482777624689997}

In [409]:
getWeights(df,entity_list,method="unlikeability",normalized = True)

{'REGION': 0.08336379093899475,
 'ACCOUNTCALLEE': 0.0,
 'ACCOUNTCALLER': 0.0,
 'API': 0.27753499976163615,
 'CALLTYPE': 0.05434696244974494,
 'RECIPIENTACCOUNTALIAS': 0.0,
 'SERVICE': 0.1334837674204262,
 'SOURCE': 0.20667736655595986,
 'USERNAME': 0.24459311287323807}

### Entire dataset for June

In [410]:
df[(df['START_TIME'] > "2019-06-01") & (df['START_TIME'] < "2019-07-01") ][['ID','ID_B','START_TIME']+entity_list].head(50)

,ID,ID_B,START_TIME,REGION,ACCOUNTCALLEE,ACCOUNTCALLER,API,CALLTYPE,RECIPIENTACCOUNTALIAS,SERVICE,SOURCE,USERNAME
1526,4820,45054179,2019-06-19 09:00:00.000 -0700,"us-east-1""""","249446771485""""","249446771485""""","ListAccessKeys""""","AwsApiCall""""","lacework-test""""","iam.amazonaws.com""""","104.11.163.140""""","IAMUser/249446771485:dev_swathi"""""
1527,4821,45054179,2019-06-19 09:00:00.000 -0700,"us-east-1""""","249446771485""""","249446771485""""","ListAccessKeys""""","AwsApiCall""""","lacework-test""""","iam.amazonaws.com""""","104.11.163.140""""","IAMUser/249446771485:dev_swathi"""""
1528,4804,45011627,2019-06-19 05:00:00.000 -0700,"us-east-1""""","249446771485""""","249446771485""""","AssumeRole""""","AwsApiCall""""","lacework-test""""","sts.amazonaws.com""""","35.164.9.186""""",AssumedRole/249446771485:tf-qa9-kiam-server-ro...
1529,4803,45011627,2019-06-19 05:00:00.000 -0700,"us-east-1""""","249446771485""""","249446771485""""","AssumeRole""""","AwsApiCall""""","lacework-test""""","sts.amazonaws.com""""","35.164.9.186""""",AssumedRole/249446771485:tf-qa9-kiam-server-ro...
1530,4803,45011628,2019-06-19 05:00:00.000 -0700,"us-east-1""""","249446771485""""","249446771485""""","AssumeRole""""","AwsApiCall""""","lacework-test""""","sts.amazonaws.com""""","35.164.9.186""""","AssumedRole/249446771485:masters.qa9.k8s.local"""""
1531,4805,45011628,2019-06-19 05:00:00.000 -0700,"us-east-1""""","249446771485""""","249446771485""""","AssumeRole""""","AwsApiCall""""","lacework-test""""","sts.amazonaws.com""""","35.164.9.186""""","AssumedRole/249446771485:masters.qa9.k8s.local"""""
1532,4804,45011629,2019-06-19 05:00:00.000 -0700,"us-east-1""""","249446771485""""","249446771485""""","AssumeRole""""","AwsApiCall""""","lacework-test""""","sts.amazonaws.com""""","35.164.9.186""""",AssumedRole/249446771485:tf-qa9-kiam-server-ro...
1533,4803,45011629,2019-06-19 05:00:00.000 -0700,"us-east-1""""","249446771485""""","249446771485""""","AssumeRole""""","AwsApiCall""""","lacework-test""""","sts.amazonaws.com""""","35.164.9.186""""",AssumedRole/249446771485:tf-qa9-kiam-server-ro...
1534,4804,45011675,2019-06-19 05:00:00.000 -0700,"us-east-1""""","249446771485""""","249446771485""""","AssumeRole""""","AwsApiCall""""","lacework-test""""","sts.amazonaws.com""""","35.164.9.186""""",AssumedRole/249446771485:tf-qa9-kiam-server-ro...
1535,4803,45011675,2019-06-19 05:00:00.000 -0700,"us-east-1""""","249446771485""""","249446771485""""","AssumeRole""""","AwsApiCall""""","lacework-test""""","sts.amazonaws.com""""","35.164.9.186""""",AssumedRole/249446771485:tf-qa9-kiam-server-ro...


In [411]:
#Counter(df[(df['START_TIME'] > "2019-06-01") & (df['START_TIME'] < "2019-07-01") ]['START_TIME'])

In [412]:
getWeights(df,entity_list,method="entropy",start_time = "2019-06-01", end_time = "2019-07-01", normalized = False)

{'REGION': 1.8597972633945228,
 'ACCOUNTCALLEE': -0.0,
 'ACCOUNTCALLER': 0.047220392303659,
 'API': 4.432701062305982,
 'CALLTYPE': 0.08427102959240439,
 'RECIPIENTACCOUNTALIAS': -0.0,
 'SERVICE': 2.6874009392959755,
 'SOURCE': 2.9272742761883186,
 'USERNAME': 3.2607397378473}

In [413]:
getWeights(df,entity_list,method="entropy",start_time = "2019-06-01", end_time = "2019-07-01")

{'REGION': 0.12156010640608095,
 'ACCOUNTCALLEE': -0.0,
 'ACCOUNTCALLER': 0.0030864202383504694,
 'API': 0.289730296632853,
 'CALLTYPE': 0.005508124743395535,
 'RECIPIENTACCOUNTALIAS': -0.0,
 'SERVICE': 0.17565395463608724,
 'SOURCE': 0.19133256054143927,
 'USERNAME': 0.21312853680179347}

In [414]:
getWeights(df,entity_list,method="unlikeability",start_time = "2019-06-01", end_time = "2019-07-01", normalized = False)

{'REGION': 0.7163585672746298,
 'ACCOUNTCALLEE': 0.0,
 'ACCOUNTCALLER': 0.016131823599986772,
 'API': 0.9790926386830451,
 'CALLTYPE': 0.031551148411015584,
 'RECIPIENTACCOUNTALIAS': 0.0,
 'SERVICE': 0.9124009676440448,
 'SOURCE': 0.8994004097926089,
 'USERNAME': 0.9426235443045793}

In [415]:
getWeights(df,entity_list,method="unlikeability",start_time = "2019-06-01", end_time = "2019-07-01")

{'REGION': 0.15927718822434916,
 'ACCOUNTCALLEE': 0.0,
 'ACCOUNTCALLER': 0.003586795246565468,
 'API': 0.21769422412841577,
 'CALLTYPE': 0.007015171498924964,
 'RECIPIENTACCOUNTALIAS': 0.0,
 'SERVICE': 0.20286580952385797,
 'SOURCE': 0.1999752287525515,
 'USERNAME': 0.20958558262533514}

### Use 2 months' data, but focus on a particular ID only

In [416]:
df.loc[(df['START_TIME'] > "2019-05-01") & (df['START_TIME'] < "2019-07-01")& (df['ID'] == 2796)]

,ID,REGION,ACCOUNTCALLEE,ACCOUNTCALLER,API,CALLTYPE,RECIPIENTACCOUNTALIAS,SERVICE,SOURCE,START_TIME,END_TIME,USERNAME,ID_B,AWS_REGION,EVENT_NAME
0,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketAcl""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.28.200""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529589,us-west-2,GetBucketAcl
1,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPublicAccessBlock""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.25.120""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529590,us-west-2,GetBucketPublicAccessBlock
2,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPolicyStatus""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.28.200""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529591,us-west-2,GetBucketPolicyStatus
3,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPolicyStatus""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.25.120""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529592,us-west-2,GetBucketPolicyStatus
4,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPublicAccessBlock""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.28.200""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529594,us-west-2,GetBucketPublicAccessBlock
5,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPolicyStatus""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.25.120""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529595,us-west-2,GetBucketPolicyStatus
6,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketAcl""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.25.120""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529596,us-west-2,GetBucketAcl
7,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPublicAccessBlock""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.28.200""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529597,us-west-2,GetBucketPublicAccessBlock
8,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPublicAccessBlock""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.54.225""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529598,us-west-2,GetBucketPublicAccessBlock
9,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketAcl""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.57.122""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529599,us-west-2,GetBucketAcl


In [417]:
getData(df,'API', id_val = 2796, start_time = "2019-05-01", end_time = "2019-07-01")

array(['GetBucketAcl""', 'GetBucketPublicAccessBlock""',
       'GetBucketPolicyStatus""', 'GetBucketPolicyStatus""',
       'GetBucketPublicAccessBlock""', 'GetBucketPolicyStatus""',
       'GetBucketAcl""', 'GetBucketPublicAccessBlock""',
       'GetBucketPublicAccessBlock""', 'GetBucketAcl""', 'GetBucketAcl""',
       'GetBucketPolicyStatus""', 'GetBucketPublicAccessBlock""',
       'GetBucketPolicyStatus""', 'GetBucketPolicyStatus""',
       'GetBucketPublicAccessBlock""', 'GetBucketPolicyStatus""',
       'GetBucketPublicAccessBlock""', 'GetBucketPolicyStatus""',
       'GetBucketAcl""', 'GetBucketPublicAccessBlock""', 'GetBucketAcl""',
       'GetBucketPublicAccessBlock""', 'GetBucketPolicyStatus""',
       'GetBucketPolicyStatus""', 'GetBucketPublicAccessBlock""',
       'GetBucketPolicyStatus""', 'GetBucketAcl""',
       'GetBucketPublicAccessBlock""', 'GetBucketPublicAccessBlock""',
       'GetBucketAcl""', 'GetBucketAcl""', 'GetBucketPolicyStatus""',
       'GetBucketPublicAc

In [418]:
df[(df['START_TIME'] >= start_time) &  (df['START_TIME'] < end_time) & (df['ID'] == 2796)]

,ID,REGION,ACCOUNTCALLEE,ACCOUNTCALLER,API,CALLTYPE,RECIPIENTACCOUNTALIAS,SERVICE,SOURCE,START_TIME,END_TIME,USERNAME,ID_B,AWS_REGION,EVENT_NAME
0,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketAcl""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.28.200""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529589,us-west-2,GetBucketAcl
1,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPublicAccessBlock""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.25.120""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529590,us-west-2,GetBucketPublicAccessBlock
2,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPolicyStatus""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.28.200""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529591,us-west-2,GetBucketPolicyStatus
3,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPolicyStatus""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.25.120""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529592,us-west-2,GetBucketPolicyStatus
4,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPublicAccessBlock""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.28.200""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529594,us-west-2,GetBucketPublicAccessBlock
5,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPolicyStatus""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.25.120""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529595,us-west-2,GetBucketPolicyStatus
6,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketAcl""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.25.120""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529596,us-west-2,GetBucketAcl
7,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPublicAccessBlock""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.28.200""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529597,us-west-2,GetBucketPublicAccessBlock
8,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketPublicAccessBlock""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.54.225""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529598,us-west-2,GetBucketPublicAccessBlock
9,2796,"us-west-2""""","249446771485""""","249446771485""""","GetBucketAcl""""","AwsApiCall""""","lacework-test""""","s3.amazonaws.com""""","172.16.57.122""""",2019-05-01 12:00:00.000 -0700,2019-05-01 13:00:00.000 -0700,"IAMUser/249446771485:dev_quentin""""",28529599,us-west-2,GetBucketAcl


In [419]:
getWeights(df,entity_list,id_val=2796,method="entropy",start_time = "2019-05-01", end_time = "2019-07-01", normalized = False)

{'REGION': 0.7356219397587946,
 'ACCOUNTCALLEE': -0.0,
 'ACCOUNTCALLER': -0.0,
 'API': 1.0821955300387671,
 'CALLTYPE': -0.0,
 'RECIPIENTACCOUNTALIAS': -0.0,
 'SERVICE': -0.0,
 'SOURCE': 1.760175526826092,
 'USERNAME': -0.0}

In [420]:
getWeights(df,entity_list,id_val=2796,method="entropy",start_time = "2019-05-01", end_time = "2019-07-01", normalized = True)

{'REGION': 0.2055962491969545,
 'ACCOUNTCALLEE': -0.0,
 'ACCOUNTCALLER': -0.0,
 'API': 0.30245881729225654,
 'CALLTYPE': -0.0,
 'RECIPIENTACCOUNTALIAS': -0.0,
 'SERVICE': -0.0,
 'SOURCE': 0.49194493351078894,
 'USERNAME': -0.0}

In [421]:
getWeights(df,entity_list,id_val=2796,method="unlikeability",start_time = "2019-05-01", end_time = "2019-07-01", normalized = False)

{'REGION': 0.4148936170212766,
 'ACCOUNTCALLEE': 0.0,
 'ACCOUNTCALLER': 0.0,
 'API': 0.6702127659574468,
 'CALLTYPE': 0.0,
 'RECIPIENTACCOUNTALIAS': 0.0,
 'SERVICE': 0.0,
 'SOURCE': 0.8404255319148937,
 'USERNAME': 0.0}

In [422]:
getWeights(df,entity_list,id_val=2796,method="unlikeability",start_time = "2019-05-01", end_time = "2019-07-01", normalized = True)

{'REGION': 0.21546961325966849,
 'ACCOUNTCALLEE': 0.0,
 'ACCOUNTCALLER': 0.0,
 'API': 0.34806629834254144,
 'CALLTYPE': 0.0,
 'RECIPIENTACCOUNTALIAS': 0.0,
 'SERVICE': 0.0,
 'SOURCE': 0.43646408839779005,
 'USERNAME': 0.0}